In [1]:
# INSTALLING LIBRARIES AND DEPENDENCIES:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# DOWNLOADING THE DATASET:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [4]:
# READING THE DATASET AND INSPECTING:
with open("./input.txt", "r", encoding="utf-8") as f:
    text = f.read()

# NUMBER OF CHARACTERS:
print("Number of characters:", len(text))

Number of characters: 1115394


In [ ]:
# INSPECTION:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [5]:
# GETTING ALL THE UNIQUE CHARACTERS:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# CHARACTER LEVEL ENCODING:
s2i = { ch:i for i, ch in enumerate(chars) }
i2s = { i:ch for i, ch in enumerate(chars) }
encode = lambda x: [s2i[c] for c in x]
decode = lambda y: ''.join([i2s[i] for i in y])

# ENCODING AND DECODING:
print(encode("Hello, I am Thinam"))
print(decode(encode("Hello, I am Thinam")))

[20, 43, 50, 50, 53, 6, 1, 21, 1, 39, 51, 1, 32, 46, 47, 52, 39, 51]
Hello, I am Thinam


In [7]:
# ENCODING THE ENTIRE DATASET:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [8]:
# SPLITTING THE DATASET:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
# INSPECTING THE CHUNK OF DATASET:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When context is {context} the target is {target}")

When context is tensor([18]) the target is 47
When context is tensor([18, 47]) the target is 56
When context is tensor([18, 47, 56]) the target is 57
When context is tensor([18, 47, 56, 57]) the target is 58
When context is tensor([18, 47, 56, 57, 58]) the target is 1
When context is tensor([18, 47, 56, 57, 58,  1]) the target is 15
When context is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
When context is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [ ]:
# INITIALIZING BATCHES OF DATA:
torch.manual_seed(2023)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

# IMPLEMENTATION:
xb, yb = get_batch("train")
print("Inputs:")
print(xb.shape)
print(xb)
print("Targets:")
print(yb.shape)
print(yb)
print("-------------------------------------------------------------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"When context is {context} the target is {target}")

Inputs:
torch.Size([4, 8])
tensor([[45,  1, 57, 53, 59, 52, 42,  6],
        [43, 55, 59, 43, 57, 58,  0, 25],
        [ 1, 54, 53, 57, 58, 43, 42,  1],
        [41, 53, 59, 50, 42,  0, 52, 53]])
Targets:
torch.Size([4, 8])
tensor([[ 1, 57, 53, 59, 52, 42,  6,  0],
        [55, 59, 43, 57, 58,  0, 25, 63],
        [54, 53, 57, 58, 43, 42,  1, 53],
        [53, 59, 50, 42,  0, 52, 53, 58]])
-------------------------------------------------------------
When context is tensor([45]) the target is 1
When context is tensor([45,  1]) the target is 57
When context is tensor([45,  1, 57]) the target is 53
When context is tensor([45,  1, 57, 53]) the target is 59
When context is tensor([45,  1, 57, 53, 59]) the target is 52
When context is tensor([45,  1, 57, 53, 59, 52]) the target is 42
When context is tensor([45,  1, 57, 53, 59, 52, 42]) the target is 6
When context is tensor([45,  1, 57, 53, 59, 52, 42,  6]) the target is 0
When context is tensor([43]) the target is 55
When context is tensor

In [ ]:
# INSTALLING LIBRARIES: 
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(2023)


# IMPLEMENTATION OF BIGRAM LANGUAGE MODEL:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    # idx and targets are both (B, T) tensors.
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1) # (B, C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

# IMPLEMENTATION OF THE MODEL:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.7348, grad_fn=<NllLossBackward0>)

3-easo:,s3$.f$,?w:JvkHwOBib;z-?nfyHdTBm,A PDy:FJNrTiiWdJcg3g!utZby
ydSq:ztAmTFb$KG
m$.Ln!kpAHT!gHb$v


In [ ]:
# OPTIMIZER OBJECT:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
# GENERAL TRAINING LOOP:
batch_size = 32
for _ in range(10000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.4221279621124268


In [ ]:
# IMPLEMENTATION OF THE MODEL:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


RITI he iohat bl's h, avit thal gr:

LBumsery in.

QQindulllod, bllot d, ve s inchaceryou wKas
Jaiclliso iathre,
youeneranir dr, as Contoutlle, attimy inven w it wopr;
OTion my, Sorerwe s th! lie HOR:'s wed ctacavephanacuto nd

Wirs tr's ierd?
I'e, n,x ' ars!
WI d 's hus czds tZAng werofDUTound ithit totond:
TENGHot. rr. pise, s:

A:
Giar t herenod mpundy; promelavenan, rouchathagllknthonr;

ALA a


In [9]:
# SELF-ATTENTION:
torch.manual_seed(2023)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

head_size = 16
query = nn.Linear(C, head_size, bias=False)
key = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)    # (B,T,16)
q = query(x)  # (B,T,16)
v = value(x)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [12]:
# BATCH NORMALIZATION:
class LayerNorm:

    def __init__(self, dim, eps=1e-5):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
    
    def __call__(self, x):
        xmean = x.mean(1, keepdim=True)
        xvar = x.var(1, keepdim=True)
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

# IMPLEMENtATION:
module = LayerNorm(100)
x = torch.randn(32, 100)
out = module(x)
print(out.shape)

torch.Size([32, 100])


In [13]:
# INSPECTION:
out[0, :].mean(), out[0, :].std()

(tensor(-5.9605e-09), tensor(1.0000))